# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [6]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [7]:
inputs_shape= IMG_SIZE=(80, 80, 3)
BATCH_SIZE = 32
latent_dim = 100
intermediate_dim = 100
TRAIN_BUF = 600
TEST_BUF = 100

In [8]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [9]:
import tensorflow as tf

In [10]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)


In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [12]:
_instance_scale

1.0

In [13]:
import numpy as np
from collections.abc import Iterable

In [14]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
_outputs_shape

19200

# Model's Layers definition

In [16]:
_uf=100
enc_lays = [
    tf.keras.layers.Conv2D(filters=_uf, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=_uf*9, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=_uf*3*3, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(20 , 20, _uf)),
    tf.keras.layers.Conv2DTranspose(filters=_uf, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=_uf*3, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition

In [17]:
model_name = dataset_name+'ConvAE'
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [18]:
from training.traditional.autoencoders.autoencoder import autoencoder as AE

In [19]:
inputs_shape=IMG_SIZE

In [20]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [21]:
from utils.data_and_files.file_utils import create_if_not_exist

In [22]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [23]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonConvAE\\var_save_dir'

In [24]:
#to restore trained model, set filepath=_restore

In [25]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 80, 80, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 39, 39, 100)       2800      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 19, 900)       810900    
_________________________________________________________________
flatten (Flatten)            (None, 324900)            0         
_________________________________________________________________
dense (Dense)                (None, 100)               32490100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 900)               90900     
_________________________________________________________________
reshape (Reshape)            (None, 20, 20, 100)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 40, 40, 100)       90100     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 80, 80, 300)       270300    
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 80, 80, 3)         8103      
_________________________________________________________________
batch_normalization_1 (Batch (None, 80, 80, 3)         1

  WARNING  | None


In [26]:
#ae.compile(metrics=None)
ae.compile()

Model: "pokemonConvAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 80, 80, 3)]       0         
_________________________________________________________________
inference (Model)            (None, 100)               33304200  
_________________________________________________________________
generative (Model)           (None, 80, 80, 3)         459415    
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 80, 80, 3)]       0         
Total params: 33,763,615
Trainable params: 33,763,409
Non-trainable params: 206
_________________________________________________________________
None


In [27]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [28]:
progbar = NotebookPrograssBar(leave_outer=False)

In [29]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [30]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [31]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonConvAE\\csv_dir\\pokemonConvAE.csv'

In [32]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [33]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [34]:
import numpy as np

In [35]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=50,
    epochs=100, 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)

ValueError: in converted code:

    ..\..\graphs\basics\AE_graph.py:52 generate_sample  *
        generated = decode_fn(model=model, latent=eps, inputs_shape=inputs_shape, apply_sigmoid=True)
    ..\..\graphs\basics\AE_graph.py:42 decode_fn  *
        x_logits = model('generative', [latent])
    ..\..\training\traditional\autoencoders\autoencoder.py:129 get_varibale  *
        return self.get_variables()[var_name](*param)
    C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:778 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\tensorflow_core\python\keras\engine\network.py:717 call
        convert_kwargs_to_constants=base_layer_utils.call_context().saving)
    C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\tensorflow_core\python\keras\engine\network.py:891 _run_internal_graph
        output_tensors = layer(computed_tensors, **kwargs)
    C:\ProgramData\Ana